In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

train = pd.read_csv("/kaggle/input/comment-category-prediction-challenge/train.csv")
test = pd.read_csv("/kaggle/input/comment-category-prediction-challenge/test.csv")

/kaggle/input/comment-category-prediction-challenge/Sample.csv
/kaggle/input/comment-category-prediction-challenge/train.csv
/kaggle/input/comment-category-prediction-challenge/test.csv


In [2]:
# ─────────────────────────────────────────────────────────────────────────────
# 0.  IMPORTS
# ─────────────────────────────────────────────────────────────────────────────
import gc
import copy
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from scipy.sparse import issparse, csr_matrix

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, f1_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import ComplementNB
from sklearn.ensemble import VotingClassifier, BaggingClassifier, StackingClassifier
from sklearn.calibration import CalibratedClassifierCV

try:
    from xgboost import XGBClassifier
    HAS_XGB = True
except ImportError:
    HAS_XGB = False; print("⚠  xgboost not installed")

try:
    from lightgbm import LGBMClassifier
    HAS_LGBM = True
except ImportError:
    HAS_LGBM = False; print("⚠  lightgbm not installed")

try:
    import optuna
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    HAS_OPTUNA = True
except ImportError:
    HAS_OPTUNA = False; print("⚠  optuna not installed — skipping tuning")

In [3]:
# ─────────────────────────────────────────────────────────────────────────────
# 1.  CONFIG  ← tune these to trade accuracy for memory
# ─────────────────────────────────────────────────────────────────────────────
TFIDF_MAX_FEATURES = 15_000   # 30k → OOM; 10-15k is usually fine for F1
TFIDF_NGRAM        = (1, 2)
TUNING_TRIALS      = 15       # Optuna trials per model
K_SHORTLIST        = 3        # top-K models to tune & ensemble
RANDOM_STATE       = 42
CV_FOLDS           = 2


# ─────────────────────────────────────────────────────────────────────────────
# 2.  LOAD DATA
# ─────────────────────────────────────────────────────────────────────────────
train_raw = pd.read_csv("/kaggle/input/comment-category-prediction-challenge/train.csv")
test_raw  = pd.read_csv("/kaggle/input/comment-category-prediction-challenge/test.csv")


# ─────────────────────────────────────────────────────────────────────────────
# 3.  FEATURE ENGINEERING
# ─────────────────────────────────────────────────────────────────────────────
IDENTITY_CATS = ["race", "religion", "gender"]

def engineer_features(df: pd.DataFrame, ref_cols=None) -> pd.DataFrame:
    df = df.copy()
    df = df.drop(columns=["post_id"], errors="ignore")

    df["created_date"] = pd.to_datetime(df["created_date"])
    df["hour"]         = df["created_date"].dt.hour.astype("int8")
    df["dayofweek"]    = df["created_date"].dt.dayofweek.astype("int8")
    df["is_weekend"]   = df["dayofweek"].isin([5, 6]).astype("int8")
    df = df.drop(columns=["created_date"])

    df[IDENTITY_CATS] = df[IDENTITY_CATS].fillna("none")
    df = pd.get_dummies(df, columns=IDENTITY_CATS, drop_first=False, dtype="int8")

    # align to training schema
    if ref_cols is not None:
        for col in ref_cols:
            if col not in df.columns:
                df[col] = 0
        df = df[ref_cols]

    return df


train_clean   = train_raw.dropna(subset=["comment"]).reset_index(drop=True)
features      = engineer_features(train_clean.drop(columns=["label"]))
labels        = train_clean["label"].copy()
test_features = engineer_features(test_raw, ref_cols=features.columns.tolist())

# free raw frames — we no longer need them
del train_raw
gc.collect()

25

In [4]:
# ─────────────────────────────────────────────────────────────────────────────
# 4.  COLUMN GROUPS
# ─────────────────────────────────────────────────────────────────────────────
TEXT_COL      = "comment"
IDENTITY_COLS = [c for c in features.columns if c.startswith(("race_", "religion_", "gender_"))]
REACTION_COLS = ["upvote", "downvote", "emoticon_1", "emoticon_2", "emoticon_3"]
IF_COLS       = ["if_1", "if_2"]
TIME_COLS     = ["hour", "dayofweek", "is_weekend"]
NUM_CLASSES   = labels.nunique()

In [5]:
# ─────────────────────────────────────────────────────────────────────────────
# 5.  PREPROCESSOR FACTORY
#     sparse_threshold=1.0  → ALWAYS keeps sparse output (critical!)
#     dtype=float32         → halves RAM vs float64
# ─────────────────────────────────────────────────────────────────────────────
def make_tfidf():
    return TfidfVectorizer(
        lowercase=True, stop_words="english",
        max_features=TFIDF_MAX_FEATURES, ngram_range=TFIDF_NGRAM,
        min_df=5, max_df=0.9, dtype=np.float32)


def make_preprocessor():
    return ColumnTransformer(
        transformers=[
            ("text",     make_tfidf(),           TEXT_COL),
            ("identity", "passthrough",           IDENTITY_COLS),
            ("reaction", MinMaxScaler(),          REACTION_COLS),
            ("ifs",      MinMaxScaler(),          IF_COLS),
            ("time",     MinMaxScaler(),          TIME_COLS),
        ],
        sparse_threshold=1.0,   # ← NEVER densify
        n_jobs=1,
    )


def make_pipe():
    return Pipeline([("pre", make_preprocessor())])

In [6]:
# ─────────────────────────────────────────────────────────────────────────────
# 6.  TRAIN / VAL SPLIT
# ─────────────────────────────────────────────────────────────────────────────
idx = np.arange(len(features))
idx_tr, idx_val, y_tr, y_val = train_test_split(
    idx, labels.values,
    test_size=0.2, stratify=labels.values, random_state=RANDOM_STATE
)
X_tr  = features.loc[idx_tr]
X_val = features.loc[idx_val]

# Fit preprocessor ONCE and keep sparse matrices
_pipe = make_pipe()
Xtr_full  = _pipe.fit_transform(X_tr)   # scipy sparse, float32
Xval_full = _pipe.transform(X_val)      # scipy sparse, float32
print(f"✓ Preprocessed  train: {Xtr_full.shape}  val: {Xval_full.shape}")
print(f"  Train sparse nnz: {Xtr_full.nnz:,}  (~{Xtr_full.nnz*4/1e6:.0f} MB)")

✓ Preprocessed  train: (158399, 15029)  val: (39600, 15029)
  Train sparse nnz: 4,641,354  (~19 MB)


In [7]:
# ─────────────────────────────────────────────────────────────────────────────
# 7.  SPARSE → DENSE HELPER  (only called for tree-based models)
#     Converts in float32, then deletes the dense copy after use
# ─────────────────────────────────────────────────────────────────────────────
def to_dense_f32(X):
    """Convert sparse to float32 dense array. Caller must del result + gc."""
    arr = X.toarray().astype(np.float32) if issparse(X) else X.astype(np.float32)
    return arr

DENSE_MODELS = {"XGBoost", "RandomForest"}


# ─────────────────────────────────────────────────────────────────────────────
# 8.  MODEL REGISTRY  ← ADD / REMOVE MODELS HERE
# ─────────────────────────────────────────────────────────────────────────────
MODEL_REGISTRY = {
    "LogisticRegression": LogisticRegression(
        C=1.0, max_iter=2000, solver="saga", n_jobs=-1),

    # CalibratedClassifierCV wraps LinearSVC → gives predict_proba for ensembles
    "LinearSVC": CalibratedClassifierCV(
        LinearSVC(C=1.0, class_weight="balanced", max_iter=5000)),

    # ComplementNB: sparse-native, great for imbalanced text
    "ComplementNB": ComplementNB(alpha=0.1),
}

if HAS_XGB:
    MODEL_REGISTRY["XGBoost"] = XGBClassifier(
        objective="multi:softmax", num_class=NUM_CLASSES,
        n_estimators=300, max_depth=6, learning_rate=0.1,
        subsample=0.8, colsample_bytree=0.8,
        tree_method="hist", device="cpu",
        eval_metric="mlogloss", n_jobs=-1, random_state=RANDOM_STATE)

if HAS_LGBM:
    MODEL_REGISTRY["LightGBM"] = LGBMClassifier(
        n_estimators=300, num_leaves=63, learning_rate=0.05,
        subsample=0.8, colsample_bytree=0.8,
        n_jobs=-1, random_state=RANDOM_STATE, verbose=-1)

# ── To add a new model, just append here: ──────────────────────────────
# from sklearn.linear_model import SGDClassifier
# MODEL_REGISTRY["SGD"] = SGDClassifier(loss="modified_huber", n_jobs=-1)
# ───────────────────────────────────────────────────────────────────────

In [8]:
# ─────────────────────────────────────────────────────────────────────────────
# 9.  EVALUATION HELPERS
# ─────────────────────────────────────────────────────────────────────────────
def evaluate(clf, X_v, y_v, name="", verbose=True):
    preds    = clf.predict(X_v)
    macro_f1 = f1_score(y_v, preds, average="macro")
    if verbose:
        print(f"\n{name}")
        print(classification_report(y_v, preds, zero_division=0))
    return macro_f1

def fit_eval(name, clf, Xtr, Xv, ytr, yv, verbose=True):
    needs_dense = any(tag in name for tag in DENSE_MODELS)

    if needs_dense:
        Xtr_in = to_dense_f32(Xtr)
        clf.fit(Xtr_in, ytr)
        del Xtr_in; gc.collect()
        Xv_in = to_dense_f32(Xv)
        f1 = evaluate(clf, Xv_in, yv, name=name, verbose=verbose)
        del Xv_in; gc.collect()

    elif name == "LightGBM":
        import lightgbm as lgb
        clf.fit(Xtr, ytr,
                eval_set=[(Xv, yv)],
                callbacks=[lgb.early_stopping(40, verbose=False),
                           lgb.log_evaluation(period=-1)])
        f1 = evaluate(clf, Xv, yv, name=name, verbose=verbose)
        print(f"  → Best n_estimators found: {clf.best_iteration_}")

    else:
        clf.fit(Xtr, ytr)
        f1 = evaluate(clf, Xv, yv, name=name, verbose=verbose)

    return clf, f1

In [9]:
# ─────────────────────────────────────────────────────────────────────────────
# 10. EVALUATE ALL MODELS
# ─────────────────────────────────────────────────────────────────────────────
results = {}

print("\n" + "═"*60)
print("  FULL FEATURE SET — ALL MODELS")
print("═"*60)

for name, clf in MODEL_REGISTRY.items():
    print(f"\n▶  {name}")
    trained, score = fit_eval(name, copy.deepcopy(clf),
                              Xtr_full, Xval_full, y_tr, y_val)
    results[name] = {"score": score, "model": trained}
    gc.collect()

ranked    = sorted(results.items(), key=lambda x: x[1]["score"], reverse=True)
shortlist = [n for n, _ in ranked[:K_SHORTLIST]]

print("\n" + "═"*60)
print("  MACRO-F1 LEADERBOARD")
print("═"*60)
for i, (n, info) in enumerate(ranked, 1):
    tag = " ✓" if n in shortlist else ""
    print(f"  {i}. {n:<25} {info['score']:.4f}{tag}")
print(f"\n  Shortlisted: {shortlist}")


════════════════════════════════════════════════════════════
  FULL FEATURE SET — ALL MODELS
════════════════════════════════════════════════════════════

▶  LogisticRegression

LogisticRegression
              precision    recall  f1-score   support

           0       0.80      0.92      0.85     22834
           1       0.72      0.61      0.66      3184
           2       0.82      0.67      0.74     12488
           3       0.68      0.31      0.43      1094

    accuracy                           0.80     39600
   macro avg       0.76      0.63      0.67     39600
weighted avg       0.80      0.80      0.79     39600


▶  LinearSVC

LinearSVC
              precision    recall  f1-score   support

           0       0.80      0.91      0.85     22834
           1       0.73      0.58      0.65      3184
           2       0.80      0.68      0.73     12488
           3       0.70      0.28      0.40      1094

    accuracy                           0.79     39600
   macro avg    

In [11]:

import pickle, os

SAVE_DIR = "/kaggle/working/saved_models"
os.makedirs(SAVE_DIR, exist_ok=True)


# Add this at the end of section 10, after computing Xtr_full / Xval_full
from scipy.sparse import save_npz, load_npz

save_npz(f"{SAVE_DIR}/Xtr_full.npz", Xtr_full)
save_npz(f"{SAVE_DIR}/Xval_full.npz", Xval_full)
np.save(f"{SAVE_DIR}/y_tr.npy", y_tr)
np.save(f"{SAVE_DIR}/y_val.npy", y_val)
print("💾 Preprocessed matrices saved")

💾 Preprocessed matrices saved


In [12]:
from scipy.sparse import load_npz

_xtr_path = f"{SAVE_DIR}/Xtr_full.npz"
if os.path.exists(_xtr_path):
    Xtr_full  = load_npz(f"{SAVE_DIR}/Xtr_full.npz")
    Xval_full = load_npz(f"{SAVE_DIR}/Xval_full.npz")
    y_tr      = np.load(f"{SAVE_DIR}/y_tr.npy")
    y_val     = np.load(f"{SAVE_DIR}/y_val.npy")
    print("✅ Loaded preprocessed matrices from cache")
else:
    print("⚠ No cached matrices found — run sections 1–10 first")

✅ Loaded preprocessed matrices from cache


In [14]:
# ─────────────────────────────────────────────────────────────────────────────
# 11. HYPERPARAMETER TUNING  (RandomizedSearchCV — sklearn only)
# ─────────────────────────────────────────────────────────────────────────────
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform, randint, uniform
import pickle, os, lightgbm as lgb
from scipy.sparse import save_npz, load_npz

SAVE_DIR = "/kaggle/working/saved_models"
os.makedirs(SAVE_DIR, exist_ok=True)

# ── Save/load helpers ─────────────────────────────────────────────────────────
def save_model(name, clf, score):
    with open(f"{SAVE_DIR}/{name}.pkl", "wb") as f:
        pickle.dump({"model": clf, "score": score}, f)
    print(f"  💾 Saved {name}")

def load_model(name):
    path = f"{SAVE_DIR}/{name}.pkl"
    if os.path.exists(path):
        with open(path, "rb") as f:
            return pickle.load(f)
    return None

# ── Save preprocessed matrices (so future restarts skip section 6) ────────────
save_npz(f"{SAVE_DIR}/Xtr_full.npz", Xtr_full)
save_npz(f"{SAVE_DIR}/Xval_full.npz", Xval_full)
np.save(f"{SAVE_DIR}/y_tr.npy", y_tr)
np.save(f"{SAVE_DIR}/y_val.npy", y_val)
print("💾 Preprocessed matrices saved")

# ── Config ────────────────────────────────────────────────────────────────────
TUNING_ITER  = 15
CV_FOLDS     = 2
SKIP_TUNING  = {"XGBoost"}   # default params are good enough; saves hours

# ── LightGBM: special tuning with early stopping (faster than CV) ─────────────
def tune_lgbm(Xtr, Xv, ytr, yv, n_iter=TUNING_ITER):
    """
    Manual random search for LightGBM using a single val split + early stopping.
    Much faster than CV because bad configs die early.
    """
    from scipy.stats import randint, uniform
    import numpy as np

    rng = np.random.RandomState(RANDOM_STATE)
    best_score, best_clf = -1, None

    for i in range(n_iter):
        params = dict(
            num_leaves      = int(rng.randint(31, 127)),
            learning_rate   = float(np.exp(rng.uniform(np.log(0.03), np.log(0.2)))),
            subsample       = float(rng.uniform(0.6, 1.0)),
            colsample_bytree= float(rng.uniform(0.6, 1.0)),
        )
        clf = LGBMClassifier(
            n_estimators=500, **params,
            n_jobs=-1, random_state=RANDOM_STATE, verbose=-1)
        clf.fit(Xtr, ytr,
                eval_set=[(Xv, yv)],
                callbacks=[lgb.early_stopping(40, verbose=False),
                           lgb.log_evaluation(period=-1)])
        score = f1_score(yv, clf.predict(Xv), average="macro")
        print(f"    iter {i+1:02d}  leaves={params['num_leaves']:3d}  "
              f"lr={params['learning_rate']:.3f}  f1={score:.4f}"
              f"  trees={clf.best_iteration_}")
        if score > best_score:
            best_score, best_clf = score, clf
            print(f"    ✓ New best: {best_score:.4f}")

    return best_clf, best_score

# ── Search spaces for RandomizedSearchCV (non-LightGBM models) ───────────────
PARAM_GRIDS = {
    "LogisticRegression": {
        "C": loguniform(1e-2, 10),
    },
    "LinearSVC": {
        "base_estimator__C":            loguniform(1e-2, 10),
        "base_estimator__class_weight": ["balanced", None],
    },
    "ComplementNB": {
        "alpha": loguniform(1e-3, 2.0),
    },
    "XGBoost": {
        "n_estimators":     [100, 200, 300],
        "max_depth":        randint(3, 7),
        "learning_rate":    [0.05, 0.1, 0.2],
        "subsample":        uniform(0.6, 0.4),
        "colsample_bytree": uniform(0.6, 0.4),
    },
}

# ── Main tuning loop ──────────────────────────────────────────────────────────
tuned_models = {}

for name in shortlist:
    print(f"\n{'─'*60}\n  Tuning: {name}\n{'─'*60}")

    # check cache first
    cached = load_model(name)
    if cached is not None:
        print(f"  ✅ Loaded from cache — macro-F1: {cached['score']:.4f}")
        tuned_models[name] = cached
        continue

    # LightGBM: custom early-stopping random search
    if name == "LightGBM":
        trained, score = tune_lgbm(Xtr_full, Xval_full, y_tr, y_val)
        evaluate(trained, Xval_full, y_val, name=name, verbose=True)

    # XGBoost / anything in SKIP_TUNING: use defaults
    elif name in SKIP_TUNING:
        print("  Skipping tuning — using defaults")
        trained, score = fit_eval(name, copy.deepcopy(MODEL_REGISTRY[name]),
                                  Xtr_full, Xval_full, y_tr, y_val)

    # Everything else: RandomizedSearchCV
    else:
        needs_dense = any(tag in name for tag in DENSE_MODELS)
        Xtr_in = to_dense_f32(Xtr_full) if needs_dense else Xtr_full

        search = RandomizedSearchCV(
            estimator=copy.deepcopy(MODEL_REGISTRY[name]),
            param_distributions=PARAM_GRIDS.get(name, {}),
            n_iter=TUNING_ITER, scoring="f1_macro",
            cv=CV_FOLDS, n_jobs=1, refit=True,
            random_state=RANDOM_STATE, verbose=1, error_score="raise")
        search.fit(Xtr_in, y_tr)

        if needs_dense:
            del Xtr_in; gc.collect()

        print(f"  Best CV macro-F1: {search.best_score_:.4f}")
        print(f"  Best params:      {search.best_params_}")

        trained = search.best_estimator_
        Xv_in   = to_dense_f32(Xval_full) if needs_dense else Xval_full
        score   = evaluate(trained, Xv_in, y_val, name=name, verbose=True)
        if needs_dense:
            del Xv_in; gc.collect()

    tuned_models[name] = {"model": trained, "score": score}
    save_model(name, trained, score)
    print(f"  ✓ Val macro-F1: {score:.4f}")
    gc.collect()

print("\n" + "═"*60)
print("  TUNED SCORES")
print("═"*60)
for n, info in sorted(tuned_models.items(), key=lambda x: x[1]["score"], reverse=True):
    print(f"  {n:<25} {info['score']:.4f}")

💾 Preprocessed matrices saved

────────────────────────────────────────────────────────────
  Tuning: LightGBM
────────────────────────────────────────────────────────────
    iter 01  leaves= 82  lr=0.182  f1=0.8057  trees=65
    ✓ New best: 0.8057
    iter 02  leaves=113  lr=0.036  f1=0.8113  trees=285
    ✓ New best: 0.8113
    iter 03  leaves= 54  lr=0.103  f1=0.8072  trees=163
    iter 04  leaves= 60  lr=0.045  f1=0.8096  trees=351
    iter 05  leaves=106  lr=0.096  f1=0.8090  trees=112
    iter 06  leaves=121  lr=0.096  f1=0.8096  trees=98
    iter 07  leaves=110  lr=0.047  f1=0.8109  trees=222
    iter 08  leaves= 85  lr=0.194  f1=0.8058  trees=59
    iter 09  leaves= 37  lr=0.041  f1=0.8060  trees=500
    iter 10  leaves= 90  lr=0.087  f1=0.8103  trees=127
    iter 11  leaves= 32  lr=0.110  f1=0.8064  trees=257
    iter 12  leaves= 38  lr=0.032  f1=0.8048  trees=500
    iter 13  leaves= 34  lr=0.054  f1=0.8086  trees=493
    iter 14  leaves= 84  lr=0.032  f1=0.8120  trees=392
 

In [15]:
# ─────────────────────────────────────────────────────────────────────────────
# 12. ENSEMBLE
#     Only sparse-native models enter VotingClassifier / Stacking
#     to avoid double-materialising dense matrices.
# ─────────────────────────────────────────────────────────────────────────────
sparse_native = [(n, info["model"]) for n, info in tuned_models.items()
                 if not any(tag in n for tag in DENSE_MODELS)]
all_tuned     = [(n, info["model"]) for n, info in tuned_models.items()]

ensemble_scores = {}

# ── 12a. Soft voting ──────────────────────────────────────────────────────────
if len(sparse_native) >= 2:
    print("\n" + "═"*60 + "\n  ENSEMBLE: SOFT VOTING\n" + "═"*60)
    voter = VotingClassifier(estimators=sparse_native, voting="soft", n_jobs=1)
    voter.fit(Xtr_full, y_tr)
    voter_f1 = evaluate(voter, Xval_full, y_val, name="SoftVoting")
    ensemble_scores["SoftVoting"] = voter_f1
    print(f"  Soft-vote macro-F1: {voter_f1:.4f}")
    gc.collect()
else:
    voter = None
    print("\n⚠  Not enough sparse-native models for soft voting.")

# ── 12b. Stacking ─────────────────────────────────────────────────────────────
if len(sparse_native) >= 2:
    print("\n" + "═"*60 + "\n  ENSEMBLE: STACKING\n" + "═"*60)
    stacker = StackingClassifier(
        estimators=sparse_native,
        final_estimator=LogisticRegression(C=1.0, max_iter=2000, solver="saga"),
        cv=3, n_jobs=1, passthrough=False)
    stacker.fit(Xtr_full, y_tr)
    stack_f1 = evaluate(stacker, Xval_full, y_val, name="Stacking")
    ensemble_scores["Stacking"] = stack_f1
    print(f"  Stacking macro-F1: {stack_f1:.4f}")
    gc.collect()
else:
    stacker = None

# ── 12c. Bagging around best sparse-native model ──────────────────────────────
if sparse_native:
    best_sparse_name = max(
        (n for n in tuned_models if not any(t in n for t in DENSE_MODELS)),
        key=lambda n: tuned_models[n]["score"])
    print(f"\n" + "═"*60 + f"\n  ENSEMBLE: BAGGING ({best_sparse_name})\n" + "═"*60)
    bagger = BaggingClassifier(
        estimator=copy.deepcopy(MODEL_REGISTRY[best_sparse_name]),
        n_estimators=10, max_samples=0.8, max_features=1.0,
        n_jobs=1, random_state=RANDOM_STATE)
    bagger.fit(Xtr_full, y_tr)
    bag_f1 = evaluate(bagger, Xval_full, y_val, name="Bagging")
    ensemble_scores["Bagging"] = bag_f1
    print(f"  Bagging macro-F1: {bag_f1:.4f}")
    gc.collect()
else:
    bagger = None


════════════════════════════════════════════════════════════
  ENSEMBLE: SOFT VOTING
════════════════════════════════════════════════════════════

SoftVoting
              precision    recall  f1-score   support

           0       0.93      0.95      0.94     22834
           1       0.78      0.74      0.76      3184
           2       0.87      0.87      0.87     12488
           3       0.74      0.46      0.57      1094

    accuracy                           0.90     39600
   macro avg       0.83      0.76      0.79     39600
weighted avg       0.89      0.90      0.89     39600

  Soft-vote macro-F1: 0.7852

════════════════════════════════════════════════════════════
  ENSEMBLE: STACKING
════════════════════════════════════════════════════════════

Stacking
              precision    recall  f1-score   support

           0       0.97      0.95      0.96     22834
           1       0.80      0.76      0.78      3184
           2       0.85      0.92      0.89     12488
      

In [16]:
# ─────────────────────────────────────────────────────────────────────────────
# 13. PICK WINNER + RETRAIN ON FULL DATA
# ─────────────────────────────────────────────────────────────────────────────
all_scores = {n: info["score"] for n, info in tuned_models.items()}
all_scores.update(ensemble_scores)

best_name = max(all_scores, key=all_scores.get)
print(f"\n🏆  Winner: {best_name}  (macro-F1 = {all_scores[best_name]:.4f})")

# map name → fitted object
fitted_map = {n: info["model"] for n, info in tuned_models.items()}
if voter   is not None: fitted_map["SoftVoting"] = voter
if stacker is not None: fitted_map["Stacking"]   = stacker
if bagger  is not None: fitted_map["Bagging"]    = bagger

# ── Retrain on ALL training data ──────────────────────────────────────────────
print("\n🔄  Retraining on full training data …")
pipe_final  = make_pipe()
X_all_pre   = pipe_final.fit_transform(features)   # sparse float32
X_test_pre  = pipe_final.transform(test_features)  # sparse float32

# Determine what the winning model actually is for the retrain
if best_name in tuned_models:
    # single model — get best params from study if available
    final_clf = copy.deepcopy(fitted_map[best_name])
else:
    # ensemble — retrain its component models from scratch on full data
    # (the ensemble itself was trained on idx_tr only)
    final_clf = copy.deepcopy(fitted_map[best_name])

needs_dense = any(tag in best_name for tag in DENSE_MODELS)

if needs_dense:
    X_all_dense  = to_dense_f32(X_all_pre);  del X_all_pre;  gc.collect()
    X_test_dense = to_dense_f32(X_test_pre); del X_test_pre; gc.collect()
    final_clf.fit(X_all_dense, labels)
    test_preds = final_clf.predict(X_test_dense)
    del X_all_dense, X_test_dense; gc.collect()
else:
    final_clf.fit(X_all_pre, labels)
    test_preds = final_clf.predict(X_test_pre)
    del X_all_pre, X_test_pre; gc.collect()


🏆  Winner: LightGBM  (macro-F1 = 0.8120)

🔄  Retraining on full training data …


In [17]:
# ─────────────────────────────────────────────────────────────────────────────
# 14. SUBMISSION
# ─────────────────────────────────────────────────────────────────────────────
submission = pd.DataFrame({
    "ID":    range(1, len(test_raw) + 1),
    "label": test_preds
})
assert len(submission) == len(test_raw), "❌ Row count mismatch!"
submission.to_csv("submission.csv", index=False)

print(f"\n✅  submission.csv saved  ({len(submission)} rows)")
print(f"   Label distribution:\n{submission['label'].value_counts().to_string()}")


✅  submission.csv saved  (102000 rows)
   Label distribution:
label
0    56942
2    34846
1     8219
3     1993
